In [20]:
import pandas as pd
import numpy as np
import nltk 
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from nltk.util import pr
from nltk.corpus import stopwords as nltk_stopwords

In [21]:
data = pd.read_csv("labeled_data.csv")
data

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [22]:
data['labels']= data['class'].map({0:"hate speech", 1:"Offensive", 2: "Normal"})
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,labels
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,Normal
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive


In [23]:
import pandas as pd

# Assuming 'data' DataFrame is already defined
data = data[['tweet', 'labels']]
print(data.head(10))

# Read the second CSV file
data2 = pd.read_csv('data2.csv')
data3 = pd.read_csv('en_dataset_with_stop_words.csv')

# Concatenate the 'tweet' columns and the 'labels' columns
concatenated_tweets = pd.concat([data['tweet'], data3['tweet']], axis=0).reset_index(drop=True)
concatenated_labels = pd.concat([data['labels'], data3['sentiment']], axis=0).reset_index(drop=True)

# Assign the concatenated tweets and labels back to the DataFrame
data = pd.DataFrame({'tweet': concatenated_tweets, 'labels': concatenated_labels})

# Print the head of the resulting DataFrame to check the result
print(data.head(10))

# Check for labels that do not match 'hate speech', 'Normal', or 'Offensive'
for label in data['labels']:
    if label not in ['hate speech', 'Normal', 'Offensive']:
        print(label)


                                               tweet     labels
0  !!! RT @mayasolovely: As a woman you shouldn't...     Normal
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  Offensive
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  Offensive
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  Offensive
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  Offensive
5  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...  Offensive
6  !!!!!!"@__BrighterDays: I can not just sit up ...  Offensive
7  !!!!&#8220;@selfiequeenbri: cause I'm tired of...  Offensive
8  " &amp; you might not get ya bitch back &amp; ...  Offensive
9  " @rhythmixx_ :hobbies include: fighting Maria...  Offensive
                                               tweet     labels
0  !!! RT @mayasolovely: As a woman you shouldn't...     Normal
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  Offensive
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  Offensive
3  !!!!!!!!! RT @C_G_Anderson: @viva_bas

In [24]:
from nltk.stem import SnowballStemmer

# Initialize stopwords and stemmer
stopwords = set(nltk_stopwords.words('english'))
stemmer = SnowballStemmer('english')


def clean(text):
    text = str(text).lower()
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'-', '', text)
    text = re.sub(r'(\w)-(\w)', r'\1\2', text)
    text = [word for word in text.split() if word not in stopwords]
    text = [stemmer.stem(word) for word in text]
    text = " ".join(text)
    return text

# Assuming 'data' is your DataFrame and 'tweet' is the column to clean
data['tweet'] = data['tweet'].apply(clean)


In [25]:
data.head()

,tweet,labels
0,rt mayasolov woman shouldnt complain clean hou...,Normal
1,rt boy dat coldtyga dwn bad cuffin dat hoe place,Offensive
2,rt urkindofbrand dawg rt ever fuck bitch start...,Offensive
3,rt cganderson vivabas look like tranni,Offensive
4,rt shenikarobert shit hear might true might fa...,Offensive


In [26]:
x= np.array(data['tweet'])
y = np.array(data['labels'])
cv = CountVectorizer()
x = cv.fit_transform(x)
x_train , x_text, y_train, y_text = train_test_split(x,y,test_size=0.05, random_state=23)
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)


DecisionTreeClassifier()

In [27]:
sample = "ass-hole"
text = re.sub(r'(\w)-(\w)', r'\1\2', sample)
print(text) # prints "asshole"
data=cv.transform([sample]).toarray()
clf.predict(data)


array(['Normal'], dtype=object)